In [4]:
import tensorflow as tf 
import tensorflow.keras as tfk
import numpy as np 
import matplotlib.pyplot as plt 

### looking at lower level training loops since we need to access whole batch for sindy OLS fit 

In [87]:
#generate artificial dataset using tensors 
import tensorflow_probability as tfp 
n = tfp.distributions.MultivariateNormalDiag([0.,0.], [1.,1.])

In [132]:
beta = tf.constant([2.,-1.]).numpy()
X = n.sample(100).numpy()
#data = tf.linalg.matvec(X,beta)
data = X@beta
data = np.expand_dims(data, -1)

In [138]:
#dataset for ols regression 
dataset_ = np.hstack((X,data))

In [139]:
dataset = tf.data.Dataset.from_tensor_slices(dataset_).batch(20)
#e = next(iter(dataset))

In [140]:
e.shape

TensorShape([20])

In [142]:
for b in dataset:
    print(np.linalg.lstsq(b.numpy()[:,0:2], b.numpy()[:,-1])[0])

[ 2. -1.]
[ 2. -1.]
[ 2. -1.]
[ 2. -1.]
[ 2. -1.]


<ipython-input-142-7febe7b89034>:2: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  print(np.linalg.lstsq(b.numpy()[:,0:2], b.numpy()[:,-1])[0])


### sindy polynomial library functions 

In [145]:
import sys
sys.path.insert(1, "../src/")
from sindy_utils import *

In [152]:
x = np.expand_dims(np.array([np.pi/2,np.pi, 0, 1]), -1)
lib = sindy_library(x, poly_order = 2, include_sine = True)
lib.shape

(4, 4)

In [153]:
#each column represents the generative functions evaluated along each axis 
lib

array([[1.00000000e+00, 1.57079633e+00, 2.46740110e+00, 1.00000000e+00],
       [1.00000000e+00, 3.14159265e+00, 9.86960440e+00, 1.22464680e-16],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 8.41470985e-01]])

In [158]:
#let's try some multidimensional data
y = np.array([[1,0], [-2,1]])
sindy_library(y, poly_order = 3, include_sine = False) #we can see the pattern i think -- yea 

array([[ 1.,  1.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 1., -2.,  1.,  4., -2.,  1., -8.,  4., -2.,  1.]])

## interfacing `train_step` with batch info 

In [160]:
#data loading 
batch_size = 32
(x_train, y_train), (x_test, y_test) = tfk.datasets.mnist.load_data()
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_test, (-1, 784))

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

In [163]:
#model 
inputs = tfk.Input(shape=(784,), name="digits")
x = tfk.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = tfk.layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = tfk.layers.Dense(10, name="predictions")(x)

model = tfk.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 digits (InputLayer)         [(None, 784)]             0         
                                                                 
 dense_1 (Dense)             (None, 64)                50240     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 predictions (Dense)         (None, 10)                650       
                                                                 
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [170]:
#train_step 
@tf.function
def train_step(data):
    x, y = data
    print(x.shape, y.shape)
    pass 

In [171]:
for step, e in enumerate(train_dataset):
    loss_value = train_step(e)

(32, 784) (32,)


In [172]:
x_train.shape

(60000, 784)